In [ ]:
! pip install gdown
! pip install pandas
! pip install seaborn
! pip install numpy
! pip install matplotlib
! pip install scikit-learn
! pip install pathlib


import gdown
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pathlib import Path

def download_from_gdrive(url, filename):
    # Extract the file ID from the URL
    file_id = url.split('/')[-2]
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Download the file
    if Path(filename).exists():
        print(f"File '{filename}' already exists. Skipping download.")
    else:
        gdown.download(download_url, filename, quiet=False)
        print(f"File downloaded as: {filename}")

train = 'https://drive.google.com/file/d/1guqSpDv1Q7ZZjSbXMYGbrTvGns0VCyU5/view?usp=drive_link'
valid = 'https://drive.google.com/file/d/1j7x8xhMimKbvW62D-XeDfuRyj9ia636q/view?usp=drive_link'
# Example usage

download_from_gdrive(train, 'train.csv')
download_from_gdrive(valid, 'valid.csv')


In [1]:
# functions attic - not used
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder , Normalizer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  
from sklearn.model_selection import train_test_split
################################################
def scale(df):
    # Normalize features in all datasets
    scaler = StandardScaler()    
    print("Scaling on xtrain")    
    return pd.DataFrame(scaler.fit_transform(df))

################################################
def normalize(df):
    # Normalize features in all datasets
    normalizer = Normalizer()
    print("Normalizing on xtrain")
    return pd.DataFrame(normalizer.fit_transform(df))
    

################################################
def fit_pipeline_data(X_train, y_train, df ):   
    numeric_features = df.select_dtypes(exclude=['object']).columns
    categorical_features = df.select_dtypes(include=['object']).columns
    
    # Pipeline including preprocessing and model training
    pipeline = Pipeline(steps=[
        ('preprocessor', ColumnTransformer(transformers=[
            ('num', SimpleImputer(strategy='mean'), numeric_features),
            ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('label_encoder', ColumnTransformer(
                    transformers=[('label', LabelEncoder(), categorical_features)],
                    remainder='passthrough'
                ))
            ]), categorical_features)
        ])),
        ('regressor', RandomForestRegressor(random_state=42))
    ])

    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)       
    return pipeline


################################################
def hyperparameter_tuning(xtrain, ytrain):
    # Initialize the Random Forest Regressor
    reg = RandomForestRegressor(n_jobs=8,random_state=43)

    # Define hyperparameters to search
    param_grid = {
        'n_estimators': [42, 100 ,200],
        'max_depth': [10, 15],
        'min_samples_split': [2, 5, 10]
    }
    # Perform grid search
    grid_search = GridSearchCV(reg, param_grid, cv=5, scoring='neg_mean_squared_error')
    print("Fitting  reg_best on xtrain_scaled, ytrain" , xtrain.shape, ytrain.shape)

    grid_search.fit(xtrain, ytrain)
    print("Fitted")
    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("Best params from hyperparametrization: " , best_params)

    # Train the model with the best hyperparameters
    # best_params = hyperparameter_tuning(xtrain, ytrain)
    # reg_best = RandomForestRegressor(**best_params)
    # reg_best.fit(xtrain_scaled, ytrain)
    # #Make predictions on all datasets
    # # 1. Make predictions on the xtrain dataset
    # print("Make predictions on the xtrain")
    # ytrain_pred = reg_best.predict(xtrain_scaled)
    # rmse = RMSE(ytrain_pred, ytrain)    
    # print(f" Best train RMSE: {rmse:.2f} | y_test.std()={ytrain.std():.2f} | y_test.mean()={ytrain.mean():.2f} ")

    # # 2. Make predictions on the xtest dataset
    # print("Make predictions on the xtest and evaluate the model")
    # ytest_pred = reg_best.predict(xtest_scaled)
    # rmse = RMSE(ytest_pred, ytest)    
    # print(f" Best test RMSE: {rmse:.2f} | y_test.std()={ytrain.std():.2f} | y_test.mean()={ytrain.mean():.2f} ")

    # # 3. Make predictions on the validation dataset
    # print("Make predictions on the validation dataset")
    # val_pred = reg_best.predict(xval_scaled)
    return best_params


In [1]:
# run here
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder , Normalizer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  
from sklearn.model_selection import train_test_split
import seaborn  as sns
import numpy as np

################################################
def RMSE(y_pred, y_true):
    return ((y_pred - y_true) ** 2).mean() ** 0.5

################################################
def train_random_forest(model , X, y=None, title="Training set"):  

    # Make predictions on the training and test sets
    y_pred = model.predict(X)
    if y is None:
        return y_pred
    
    rmse = RMSE(y_pred, y)    
    print(f" Best {title} RMSE: {rmse:.2f} | y_test.std()={y.std():.2f} | y_test.mean()={y.mean():.2f} ")        
    return rmse


################################################

def preprocess_data(df):
    # Convert date to datetime
    df2 = df.copy()
    df2['fiModelDescriptor'] = df2['fiModelDesc'].astype('str')
    df2['fiModelSeries']=df2['fiModelSeries'].astype('str')
    df2['Grouser_Tracks']=df2['Grouser_Tracks'].astype('str')
    df2['Hydraulics_Flow']=df2['Grouser_Tracks'].astype('str')

    # columns_to_keep = ['SalesID', 'SalePrice', 'saledate', 'MachineID', 'ModelID','YearMade','MachineHoursCurrentMeter','ProductSize',\
    #                   'ProductGroupDesc','fiSecondaryDesc','fiModelSeries','fiProductClassDesc','Tire_Size','fiModelDesc',]
    
    # columns_to_keep3 = ['SalesID', 'SalePrice', 'saledate', 'MachineID', 'ModelID','YearMade','MachineHoursCurrentMeter', \
    #                   'ProductGroupDesc','fiSecondaryDesc','fiModelSeries','fiProductClassDesc','Tire_Size','ProductSize','ProductGroup',\
    #                     'datasource','auctioneerID','Drive_System','Engine_Horsepower']
    
    # columns_to_keep = ['SalesID', 'SalePrice', 'saledate', 'MachineID', 'ModelID','YearMade','MachineHoursCurrentMeter','ProductSize','ProductGroup',\
    #                    'ProductGroupDesc','fiSecondaryDesc','fiBaseModel','fiModelDesc','UsageBand','state']

    columns_to_keep = ['SalesID', 'SalePrice', 'saledate', 'MachineID', 'ModelID','YearMade','MachineHoursCurrentMeter','ProductSize',\
                      'ProductGroupDesc','fiSecondaryDesc','fiModelSeries','fiProductClassDesc','Tire_Size','fiModelDesc','UsageBand']
    
    if 'SalePrice' not in df2.columns:
        columns_to_keep.remove('SalePrice')

    df2=df2[columns_to_keep]
    
    # handle dates in processing
    df2['Saledate'] = pd.to_datetime(df2.saledate)
                
    # Feature engineering with dates
    df2['SaleYear'] =  df2['Saledate'].dt.year
    #df2['SaleMonth'] =  df2['Saledate'].dt.month
    #df2['SaleDay'] =  df2['Saledate'].dt.day     # sunday sells?  
    # Replace values in YearMade column with YearFromSaledate if YearMade is before 1900 or exceeds the sale date
    df2.loc[(df2['YearMade'] <= 1900) | (df2['YearMade'] > df2['SaleYear']), 'YearMade'] = df2['SaleYear']   
    ##df2['YearMade'] = np.where(df2['YearMade'] < 1991, 1991, df2['YearMade']) 
    df2['Age'] = df2['Saledate'].dt.year - df2['YearMade']

    # machineid is a unique identifier, so we drop it
    # sale date is redundant now, so we'll drop it
    df2 = df2.drop(columns=['saledate'] , axis=1)
    df2 = df2.drop(columns=['Saledate'] , axis=1)
    df2 = df2.drop(columns=['MachineID'], axis=1)
    # df2 = df2.drop(columns=['ModelID'])  # don't touch this one !  
    # df2.drop_duplicates(inplace=True)
    # df2.fillna(df2.mode().iloc[0], inplace=True)
    
    df2 = df2.set_index('SalesID')    
    #print(sns.pairplot(df2, hue='Age'))
    return df2


# Load the original dataset 
original_dataset = pd.read_csv('train.csv')
# Preprocess the original dataset
original_dataset = preprocess_data(original_dataset)
original_dataset.drop_duplicates(inplace=True)
# Load the validation dataset 
validation_dataset = pd.read_csv('valid.csv')
# Preprocess the validation dataset
validation_dataset = preprocess_data(validation_dataset)

# Separate features and target from the original dataset
X = original_dataset.drop('SalePrice', axis=1)
y = original_dataset['SalePrice']

#From here########################################################
#X_small = X.sample(50000, random_state=43)
#X_small = X.head(50000)
X_small = X
y_small = y.loc[X_small.index]

xtrain, xtest, ytrain, ytest = train_test_split(X_small, y_small, test_size=0.3, random_state=43)
numeric_features = X.select_dtypes(exclude=['object']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# -------------------------------Impute missing values in the original and validation datasets------------------------------
mostFreqImputer  = SimpleImputer(strategy='most_frequent')
meanImputer  = SimpleImputer(strategy='mean')

xtrain[numeric_features]= meanImputer.fit_transform(xtrain[numeric_features])
xtrain[categorical_columns]= mostFreqImputer.fit_transform(xtrain[categorical_columns])

xtest[numeric_features]= meanImputer.transform(xtest[numeric_features])
xtest[categorical_columns]= mostFreqImputer.transform(xtest[categorical_columns])

validation_dataset[numeric_features] = meanImputer.transform(validation_dataset[numeric_features])
validation_dataset[categorical_columns] = mostFreqImputer.transform(validation_dataset[categorical_columns])

# -------------------------------Label encoding in the original and validation datasets------------------------------
#Combine train , test and validation datasets for label encoding to keep label consistency between'em
combined_dataset = pd.concat([xtrain, validation_dataset, xtest], ignore_index=True)
# Label encode categorical features
for col in categorical_columns:
    le = LabelEncoder()
    combined_dataset[col] = le.fit_transform(combined_dataset[col].astype(str))    
# Split back into originaltrain, test and validation datasets
xtrain = combined_dataset.iloc[:len(xtrain)]
validation_dataset_encoded = combined_dataset.iloc[len(xtrain):len(validation_dataset)+len(xtrain)]
xtest = combined_dataset.iloc[len(xtrain)+ len(validation_dataset):]
print(xtrain.shape, validation_dataset.shape, xtest.shape)
#Until here########################################################

# numeric_features = X.select_dtypes(exclude=['object']).columns
# categorical_columns = X.select_dtypes(include=['object']).columns
# mostFreqImputer  = SimpleImputer(strategy='most_frequent')
# meanImputer  = SimpleImputer(strategy='mean')
# X[numeric_features]= meanImputer.fit_transform(X[numeric_features])
# X[categorical_columns]= mostFreqImputer.fit_transform(X[categorical_columns])

# validation_dataset[numeric_features]= meanImputer.transform(validation_dataset[numeric_features])
# validation_dataset[categorical_columns]= mostFreqImputer.transform(validation_dataset[categorical_columns])

# # -------------------------------Label encoding------------------------------
# #Combine original (without SalePrice column) and validation (also) datasets for label encoding to keep consistency between train and validation datasets
# combined_dataset = pd.concat([X, validation_dataset], ignore_index=True)
# # Label encode categorical features
# for col in categorical_columns:
#     le = LabelEncoder()
#     combined_dataset[col] = le.fit_transform(combined_dataset[col])    
    
# # Split back into original and validation datasets
# X = combined_dataset.iloc[:len(X)]
# validation_dataset = combined_dataset.iloc[len(X):]
# # -------------------------------------------------------------------------------
# # Reset the index of y_original_SalePrice
# y = y.reset_index(drop=True)

# #X_small = X.sample(50000, random_state=43)
# X_small = X.head(50000)
# #X_small = X
# y_small = y.loc[X_small.index]
# xtrain, xtest, ytrain, ytest = train_test_split(X_small, y_small, test_size=0.3, random_state=43)

# Train a random forest regressor - 15, 2, 200 from hyperparameter tuning
model = RandomForestRegressor(random_state=43, max_depth=15, min_samples_split=2 , n_estimators=200)
model.fit(xtrain, ytrain)
ytrain_pred = model.predict(xtrain)   
print(f" Best train RMSE: {RMSE(ytrain_pred, ytrain):.2f} | ytrain.std()={ytrain.std():.2f} | ytrain.mean()={ytrain.mean():.2f} ")  
ytest_pred = model.predict(xtest)
print(pd.Series(
    model.feature_importances_,
    index=model.feature_names_in_
).sort_values(ascending=False))
print(f" Best test RMSE: {RMSE(ytest_pred, ytest):.2f} | y_test.std()={ytrain.std():.2f} | y_test.mean()={ytrain.mean():.2f} ")  
# Make predictions on the validation dataset
val_pred = model.predict(validation_dataset_encoded)

# Construct a series with the predictions and the SalesID as the index for submission
val_pred = pd.Series(val_pred, index=validation_dataset.index, name='SalePrice')




/tmp/ipykernel_358760/1911798532.py:86: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  original_dataset = pd.read_csv('train.csv')


(236254, 13) (11573, 13) (101253, 13)
 Best train RMSE: 7247.98 | ytrain.std()=23568.93 | ytrain.mean()=31948.03 
Age                         0.198897
fiProductClassDesc          0.155947
ProductSize                 0.141516
fiSecondaryDesc             0.136729
ModelID                     0.099702
fiModelDesc                 0.092696
YearMade                    0.068555
ProductGroupDesc            0.039190
SaleYear                    0.031073
Tire_Size                   0.021338
fiModelSeries               0.006976
MachineHoursCurrentMeter    0.006267
UsageBand                   0.001115
dtype: float64
 Best test RMSE: 8347.35 | y_test.std()=23568.93 | y_test.mean()=31948.03 


In [2]:
print(val_pred.describe())
print(val_pred.head(10))
print(validation_dataset.shape)

# create file for submission
from datetime import datetime
f'submission_{datetime.now().isoformat()}'
val_pred.to_csv(f'submission_{datetime.now().isoformat()}.csv')

count     11573.000000
mean      32364.380487
std       22559.938940
min        5426.476541
25%       14892.798202
50%       25781.012608
75%       42899.040411
max      136268.520379
Name: SalePrice, dtype: float64
SalesID
1222837    61561.696161
1222839    60520.206445
1222841    38845.932801
1222843    18647.057836
1222845    36624.787543
1222847    12535.997356
1222849    22269.094234
1222850    26377.343335
1222855    77370.541667
1222863    28710.833254
Name: SalePrice, dtype: float64
(11573, 13)
